In [1]:
from cv2 import imread, imwrite
from pyFusion.fusion import *
from pyFusion.legacy.image_fusion_tool import Image_fusion_tool
from pyFusion.models.vgg19 import VGG19
from pyFusion.models.vgg16 import VGG16
from pyFusion.models.squeezeNet import Squeeze
from pyFusion import metrics
from torch import device
from torch.cuda import is_available

import matplotlib.pyplot as plt


In [ ]:

# Parse arguments
images_path = ['images/T1-T2/t1.png', 'images/T1-T2/t2.png']
output = 'results/MRI-CT/'

# Read images
input_images = []
for image in images_path:
    input_images.append(imread(image))

In [ ]:

device = device("cuda" if is_available() else "cpu")
model = VGG19(device)
#model = VGG16(device)
#model = Squeeze(device)

FU = Fusion(input_images, model)
fused_image = FU.fuse()

In [ ]:
#imwrite("fusion", fused_image.astype('uint8'))
#plt.imshow(fused_image, cmap='gray')

In [ ]:
import numpy as np
from scipy.stats import entropy

def _pad_to(arr, shape):
    """Pad an array with trailing zeros to a given target shape.
    Parameters
    ----------
    arr : ndarray
        The input array.
    shape : tuple
        The target shape.
    Returns
    -------
    padded : ndarray
        The padded array.
    Examples
    --------
    >>> _pad_to(np.ones((1, 1), dtype=int), (1, 3))
    array([[1, 0, 0]])
    """
    if not all(s >= i for s, i in zip(shape, arr.shape)):
        raise ValueError(f'Target shape {shape} cannot be smaller than input'
                         f'shape {arr.shape} along any axis.')
    padding = [(0, s-i) for s, i in zip(shape, arr.shape)]
    return np.pad(arr, pad_width=padding, mode='constant', constant_values=0)


def normalized_mutual_information(image0, image1, *, bins=256):
    r"""Compute the normalized mutual information (NMI).
    The normalized mutual information of :math:`A` and :math:`B` is given by::
    ..math::
        Y(A, B) = \frac{H(A) + H(B)}{H(A, B)}
    where :math:`H(X) := - \sum_{x \in X}{x \log x}` is the entropy.
    It was proposed to be useful in registering images by Colin Studholme and
    colleagues [1]_. It ranges from 1 (perfectly uncorrelated image values)
    to 2 (perfectly correlated image values, whether positively or negatively).
    Parameters
    ----------
    image0, image1 : ndarray
        Images to be compared. The two input images must have the same number
        of dimensions.
    bins : int or sequence of int, optional
        The number of bins along each axis of the joint histogram.
    Returns
    -------
    nmi : float
        The normalized mutual information between the two arrays, computed at
        the granularity given by ``bins``. Higher NMI implies more similar
        input images.
    Raises
    ------
    ValueError
        If the images don't have the same number of dimensions.
    Notes
    -----
    If the two input images are not the same shape, the smaller image is padded
    with zeros.
    References
    ----------
    .. [1] C. Studholme, D.L.G. Hill, & D.J. Hawkes (1999). An overlap
           invariant entropy measure of 3D medical image alignment.
           Pattern Recognition 32(1):71-86
           :DOI:`10.1016/S0031-3203(98)00091-0`
    """
    if image0.ndim != image1.ndim:
        raise ValueError(f'NMI requires images of same number of dimensions. '
                         f'Got {image0.ndim}D for `image0` and '
                         f'{image1.ndim}D for `image1`.')
    if image0.shape != image1.shape:
        max_shape = np.maximum(image0.shape, image1.shape)
        padded0 = _pad_to(image0, max_shape)
        padded1 = _pad_to(image1, max_shape)
    else:
        padded0, padded1 = image0, image1

    hist, bin_edges = np.histogramdd(
            [np.reshape(padded0, -1), np.reshape(padded1, -1)],
            bins=bins,
            density=True,
            )

    H0 = entropy(np.sum(hist, axis=0))
    H1 = entropy(np.sum(hist, axis=1))
    H01 = entropy(np.reshape(hist, -1))

    return (H0 + H1) / H01
